# Data Collection

Here I will take information from MoMA Collection.  
Information can be found here: https://github.com/MuseumofModernArt/collection

## Import data

In [1]:
import pandas as pd
import numpy as np
import nltk
%matplotlib inline

import time
import requests

from PIL import Image
import pickle

In [2]:
pd.options.display.max_columns = None

In [3]:
df = pd.read_csv('data/MoMA_data.csv')
df.head()

,Title,Artist,ConstituentID,ArtistBio,Nationality,BeginDate,EndDate,Gender,Date,Medium,Dimensions,CreditLine,AccessionNumber,Classification,Department,DateAcquired,Cataloged,ObjectID,URL,ThumbnailURL,Circumference (cm),Depth (cm),Diameter (cm),Height (cm),Length (cm),Weight (kg),Width (cm),Seat Height (cm),Duration (sec.)
0,"Ferdinandsbrücke Project, Vienna, Austria (Ele...",Otto Wagner,6210,"(Austrian, 1841–1918)",(Austrian),(1841),(1918),(Male),1896,Ink and cut-and-pasted painted pages on paper,"19 1/8 x 66 1/2"" (48.6 x 168.9 cm)",Fractional and promised gift of Jo Carole and ...,885.1996,Architecture,Architecture & Design,1996-04-09,Y,2,http://www.moma.org/collection/works/2,http://www.moma.org/media/W1siZiIsIjU5NDA1Il0s...,NaN,NaN,NaN,48.6000,NaN,NaN,168.9000,NaN,NaN
1,"City of Music, National Superior Conservatory ...",Christian de Portzamparc,7470,"(French, born 1944)",(French),(1944),(0),(Male),1987,Paint and colored pencil on print,"16 x 11 3/4"" (40.6 x 29.8 cm)",Gift of the architect in honor of Lily Auchinc...,1.1995,Architecture,Architecture & Design,1995-01-17,Y,3,http://www.moma.org/collection/works/3,http://www.moma.org/media/W1siZiIsIjk3Il0sWyJw...,NaN,NaN,NaN,40.6401,NaN,NaN,29.8451,NaN,NaN
2,"Villa near Vienna Project, Outside Vienna, Aus...",Emil Hoppe,7605,"(Austrian, 1876–1957)",(Austrian),(1876),(1957),(Male),1903,"Graphite, pen, color pencil, ink, and gouache ...","13 1/2 x 12 1/2"" (34.3 x 31.8 cm)",Gift of Jo Carole and Ronald S. Lauder,1.1997,Architecture,Architecture & Design,1997-01-15,Y,4,http://www.moma.org/collection/works/4,http://www.moma.org/media/W1siZiIsIjk4Il0sWyJw...,NaN,NaN,NaN,34.3000,NaN,NaN,31.8000,NaN,NaN
3,"The Manhattan Transcripts Project, New York, N...",Bernard Tschumi,7056,"(French and Swiss, born Switzerland 1944)",(),(1944),(0),(Male),1980,Photographic reproduction with colored synthet...,"20 x 20"" (50.8 x 50.8 cm)",Purchase and partial gift of the architect in ...,2.1995,Architecture,Architecture & Design,1995-01-17,Y,5,http://www.moma.org/collection/works/5,http://www.moma.org/media/W1siZiIsIjEyNCJdLFsi...,NaN,NaN,NaN,50.8000,NaN,NaN,50.8000,NaN,NaN
4,"Villa, project, outside Vienna, Austria, Exter...",Emil Hoppe,7605,"(Austrian, 1876–1957)",(Austrian),(1876),(1957),(Male),1903,"Graphite, color pencil, ink, and gouache on tr...","15 1/8 x 7 1/2"" (38.4 x 19.1 cm)",Gift of Jo Carole and Ronald S. Lauder,2.1997,Architecture,Architecture & Design,1997-01-15,Y,6,http://www.moma.org/collection/works/6,http://www.moma.org/media/W1siZiIsIjEyNiJdLFsi...,NaN,NaN,NaN,38.4000,NaN,NaN,19.1000,NaN,NaN


Does not contain description data.

In [4]:
paintings = df[df.Classification == 'Painting']

In [15]:
paintings.iloc[0]['ThumbnailURL']

'http://www.moma.org/media/W1siZiIsIjE2MDU0NiJdLFsicCIsImNvbnZlcnQiLCItcmVzaXplIDMwMHgzMDBcdTAwM2UiXV0.jpg?sha=bda8976e9be36136'

In [5]:
image_id_set = paintings[['ObjectID', 'ThumbnailURL']]

In [6]:
image_id_set.iloc[0]['ThumbnailURL']

'http://www.moma.org/media/W1siZiIsIjE2MDU0NiJdLFsicCIsImNvbnZlcnQiLCItcmVzaXplIDMwMHgzMDBcdTAwM2UiXV0.jpg?sha=bda8976e9be36136'

In [7]:
resp = requests.get('http://www.moma.org/media/W1siZiIsIjE2MDU0NiJdLFsicCIsImNvbnZlcnQiLCItcmVzaXplIDMwMHgzMDBcdTAwM2UiXV0.jpg?sha=bda8976e9be36136')

In [17]:
fpath = f"data/IMAGES/MOMA_{image_id_set.iloc[0]['ObjectID']}.jpg"

In [20]:
# downloading images
def download_img(df): 
    url = str(df['ThumbnailURL'])
    if url.startswith('http'):
        try:
            resp = requests.get(url).content
        except:
            print('error retrieving')
            df['fname'] = 'error'
            return df

        fpath = f"data/IMAGES/MOMA_{df['ObjectID']}.jpg"
        print(fpath)
        with open(fpath, 'wb') as fp:
            fp.write(resp)
        df['fname'] = fpath
    else:
        df['fname'] = None
    
    pause = np.random.randint(1, 10)
    print(f'pausing for {pause} secs ...')
    time.sleep(pause) # buffer
    
    return df

In [ ]:
full_df = paintings.drop_duplicates(subset = 'ObjectID')
filename = 'pickles/raw_data_MOMA_1.pkl'
full_df.to_pickle(filename)